In [2]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
from IPython.display import display

BASE_DIR = Path.cwd().parent.parent

In [3]:
agNetXl = BASE_DIR / 'inputs/AggregateNetwork.xlsx'
nodeDf = pd.read_excel(agNetXl, sheet_name=0)
nodeDf = gpd.GeoDataFrame(nodeDf, geometry=gpd.points_from_xy(nodeDf['Longitude'], nodeDf['Latitude'], crs="EPSG:4326")).set_index('Node')
nodeDf = nodeDf.drop(columns=['Latitude', 'Longitude'])
linkDf = pd.read_excel(agNetXl, sheet_name=1)\
  .astype({'Start_Node': int, 'End_Node': int})\
  .rename(columns={'Start_Node': 'Node_fr', 'End_Node': 'Node_to'})


from shapely import LineString
def createLines(linkDf : pd.DataFrame, COL, nodeDf : gpd.GeoDataFrame, FR='_fr', TO='_to'):
  linkDf = linkDf.join(nodeDf.add_suffix(FR), on=COL + FR)
  linkDf = linkDf.join(nodeDf.add_suffix(TO), on=COL + TO)
  return linkDf.apply(lambda r:  LineString([
      [r['geometry' + FR].x, r['geometry' + FR].y],
      [r['geometry' + TO].x, r['geometry' + TO].y]
  ]), axis=1)
  


linkDf = gpd.GeoDataFrame(linkDf, geometry=createLines(linkDf, 'Node', nodeDf))
linkDf = order_link_nodes(linkDf, 'Node_fr', 'Node_to').set_index(['Node_fr', 'Node_to']).sort_index()

agNetShpXl = BASE_DIR / 'resources/shapefiles/agg_network'
agNetShpXl.mkdir(parents=True, exist_ok=True)
nodeDf.to_file( agNetShpXl / 'agg_nodes.shp')
linkDf.to_file( agNetShpXl / 'agg_links.shp')

FileNotFoundError: [Errno 2] No such file or directory: '/home/nathanczh/ctsm/faf_flow_assignment/src/inputs/AggregateNetwork.xlsx'